<a href="https://colab.research.google.com/github/pikey-msc/RiesgosFinancieros/blob/master/2020-2/Insumos/DGCF_Forwards.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
%load_ext rpy2.ipython
!rm -rf RiesgosFinancieros
!git clone "https://github.com/pikey-msc/RiesgosFinancieros/"



The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
Cloning into 'RiesgosFinancieros'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 351 (delta 28), reused 0 (delta 0), pack-reused 311
Receiving objects: 100% (351/351), 1.49 MiB | 723.00 KiB/s, done.
Resolving deltas: 100% (211/211), done.


In [52]:
%%R
#require(quantmod)
install.packages("quantmod")
library(quantmod)
# require(data.table)
install.packages("data.table")
library(data.table)

install.packages("Deriv")
library(Deriv)


library(dplyr)
con = gzcon(url('https://github.com/systematicinvestor/SIT/raw/master/sit.gz', 'rb'))
source(con)
close(con)

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/quantmod_0.4-16.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 152106 bytes (148 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[wr

In [0]:
%%R
#FUNCIÓN DE INTERPOLACIÓN ALAMBRADA

talamb=function(nodos,curva,plazos) #función de interpolación de tasas por el método alamabrada
{
  n=max(ncol(plazos),1)
  m=max(ncol(nodos),1)
  TC=matrix(0,1,n)
  TL=matrix(0,1,n)
  TF=matrix(0,1,n)
  for (j in 1:n)
  {
    i=1
    repeat
    {
      if(nodos[i]<= plazos[j] && plazos[j] <=nodos[i+1])
      {
        TC[j]=curva[i]
        TL[j]=curva[i+1]
        TF[j]=((((1+TL[j]*nodos[i+1]/360)/(1+TC[j]*nodos[i]/360))^((plazos[j]-nodos[i])/(nodos[i+1]-nodos[i]))*(1+TC[j]*nodos[i]/360))-1)*360/plazos[j]
        break
      }
      else if (plazos[j]<nodos[1])
      {
        TC[j]=curva[1]
        TL[j]=curva[1]
        TF[j]=curva[1]
        break
      }
      else if (plazos[j]>nodos[m])
      {
        TC[j]=curva[m]
        TL[j]=curva[m]
        TF[j]=curva[m]
        break
      }
      else
      {i=i+1}
    }
  }
  as.matrix(t(as.numeric(rbind(TF))))
}

#funciones necesarias
diagv=function(x)			#función para diagonalizar un vector
{
  n01=nrow(as.matrix(x))
  m01=ncol(as.matrix(x))
  dimmax=max(n01,m01)
  res=matrix(0,dimmax,dimmax)
  for (i in 1:dimmax)
  {
    res[i,i]=x[i]
  }
  res
}


diagm=function(x)		#función para diagonalizar una matriz en una de mayor dimensión
{
  n01=nrow(as.matrix(x))
  m01=ncol(as.matrix(x))
  if(m01>=n01 )
  {
    res=matrix(0,m01,m01)
    for (i in 1:(m01/n01))
    {
      z=(i-1)*n01
      for(j in 1:n01)
      {
        for(k in 1:n01)
        {
          res[z+k,z+j]=x[k,z+j]
        }
      }
    }
    res
  }
  else
  {
    "no es matriz cuadrada"
  }
}



# Definición de parámetros para valorar:
furos

Cargamos parámetros para valoración:

In [0]:
%%R
fval=as.Date("20200228",format="%Y%m%d") #Fecha de valoración
itpl=0 #poner 0 si se quiere interpolación lineal o 1 si se quiere tasa alambrada
alpha=0.98 #Nivel de confianza para obtener estimaciones de riesgo
nh=3660 #número de días de historia
yext=1 #si se usa la historia de internet o fija
#setwd(direc)	



#FORWARDS TDC
bext="RiesgosFinancieros/2020-2/Insumos/tasa_libor.txt"
bdom="RiesgosFinancieros/2020-2/Insumos/tasa_fwd.txt"
btsp="RiesgosFinancieros/2020-2/Insumos/tasa_spot.txt"
SymbolsFX_ftdc<-c("USDMXN=X", "GBPUSD=X" ) #tienen que ir en orden alfabético
plazos_fwd=cbind( 360,	9,	35,	95,	183	)
contratos_fwd=cbind(500, -290, 290, -460, 440)
kst_fwd=cbind(20.5, 19.4, 20.6, 20, 20.5)
nominal_fwd=1
yext=1 #si se carga información de yahoo en la fecha definida por fval o SymbolsFX, en caso contrario se utiliza información que se tendrá que cargar de tasas_spot.txt
trlib=1 #1 si la curva libor viene a 182 0 si no.

#FORWARDS DE IPC
#Descontamos con gubernamental
base="RiesgosFinancieros/2020-2/Insumos/tasa_guber.txt"
SymbolsEQ_find<-c("^MXX", "GCARSOA1.MX" ) #tienen que ir en orden alfabético
plazos_fwd_ind=cbind( 360,	9,	3500)
contratos_fwd_ind=cbind(5, -2, 5)
kst_fwd_ind=cbind(52000, 49500, 60000)
nominal_fwd_ind=1




# Carga de datos

In [55]:
%%R


#CARGA DE DATOS DE  FORWARDS DE TDC
#datas
#data<-read.table("tasa_tiie.txt")
data1<-read.table(bext)
data2<-read.table(bdom)

########minimos para parametrizar
n1=nrow(data1)
n2=nrow(data2)
m1_ftdc=ncol(data1)
m2_ftdc=ncol(data2)
n=min(n1,n2)-1
###NODOS###
nodos1_ftdc=data.frame(data1[1,2:m1_ftdc])
nodos2_ftdc=data.frame(data2[1,2:m2_ftdc])
####MATRICES DEL MISMO TAMAÑO MENOS DOLAR

x1_ftdc=as.data.table(mutate(data1[2:n,1:m1_ftdc],Date=as.Date(V1,format="%Y%m%d")))
x1_ftdc=x1_ftdc%>%select(-V1)
x2_ftdc=as.data.table(mutate(data2[2:n,1:m2_ftdc],Date=as.Date(V1,format="%Y%m%d")))
x2_ftdc=x2_ftdc%>%select(-V1)

###Para Dolar

if (yext==1)
{
  #Cargar los símbolos de yahoo finance para FX
  start_date=fval-3660 #fecha inicial
  
  #Creación del objeto para guardar los datos
  dataEnvFX<-new.env()
  
  #obtener los datos
  getSymbols.yahoo(SymbolsFX_ftdc,env=dataEnvFX,from=start_date, to=(fval))
  #limpiarlos, alinearnos y quedarnos con el precio de cierre 
  bt.prep(dataEnvFX,align='remove.na',fill.gaps=T)
  
  #muestra de datos
  head(dataEnvFX$prices[,2])
  
  #Nos quedamos con los precios
  X3_ftdc=data.table(Date=as.Date(index(dataEnvFX$prices[,2])),coredata(dataEnvFX$prices[,2]))
} else  
{ 
data3<-read.table(btsp)
print(head(data3))
n3<-nrow(data3)
m3<-ncol(data3)
X3=data.table(as.matrix(as.double(as.matrix(data3[2:(n+1),m3]))))
X3_find=as.data.table(mutate(data3[2:(n+1),1:m3],Date=as.Date(V1,format="%Y%m%d")))
}


# CARGA DE DATOS DE  FORWARD DE IPC

data<-read.table(base)
n<-nrow(data)
m_gov=ncol(data)
#x_orig_gov=data.frame(data[2:n,1:m_gov])
x_orig_gov=as.data.table(mutate(data[2:n,1:m_gov],Date=as.Date(V1,format="%Y%m%d")))
x_orig_gov=x_orig_gov%>%select(-V1)
nodos_gov=data.frame(data[1,2:m_gov])

  #Cargar los símbolos de yahoo finance para EQ
  start_date=fval-nh #fecha inicial
  
  #Creación del objeto para guardar los datos
  dataEnvEQ<-new.env()
  
  #obtener los datos
  getSymbols.yahoo(SymbolsEQ_find,env=dataEnvEQ,from=start_date, to=(fval))
  #limpiarlos, alinearnos y quedarnos con el precio de cierre 
  bt.prep(dataEnvEQ,align='remove.na',fill.gaps=T)
  
  #muestra de datos
#  head(dataEnvEQ$prices)
  
  #Nos quedamos con los precios
  X3_find=data.table(Date=as.Date(index(dataEnvEQ$prices[,2])),coredata(dataEnvEQ$prices[,2]))

print(head(X3_ftdc))
  print(head(X3_find))


         Date USDMXN=X
1: 2010-02-22  12.8054
2: 2010-02-23  12.9219
3: 2010-02-24  12.8023
4: 2010-02-25  12.7930
5: 2010-02-26  12.7684
6: 2010-03-01  12.7293
         Date      MXX
1: 2010-02-22 32034.34
2: 2010-02-23 31850.24
3: 2010-02-24 31788.54
4: 2010-02-25 31649.10
5: 2010-02-26 31634.54
6: 2010-03-01 31765.14


# Integración de Insumos

Aseguramiento de que todos los factores de riesgo estén alineados en sus fechas, nos quedamos con la intersección. Posteriormente, filtramos cada histórico factor de riesgo hacia esta intersección de todos.

In [0]:
%%R
#INTERSECCIÓN DE FECHAS DE TODOS LOS INSUMOS    
  
#head(x_orig_gov)  
lin_gub=data.table(Date=as.Date(X3_ftdc[x_orig_gov,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity y guber
lin_gub_bmybdst_flib=data.table(Date=as.Date(lin_gub[x1_ftdc,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor
lin_gub_bmybdst_flibfwd=data.table(Date=as.Date(lin_gub_bmybdst_flib[x2_ftdc,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd
lin_gub_bmybdst_flibfwdspind=data.table(Date=as.Date(lin_gub_bmybdst_flibfwd[X3_find,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index 


In [0]:
%%R
n=nrow(lin_gub_bmybdst_flibfwdspind) #Historia de todos


#historia de curva gubernamental
x_orig_gov=lin_gub_bmybdst_flibfwdspind[x_orig_gov,on=.(Date),nomatch=0][order(-Date)]
x_orig_gov=x_orig_gov%>%select(-Date)

#historia de curvas de forward tdc
x1_ftdc=lin_gub_bmybdst_flibfwdspind[x1_ftdc,on=.(Date),nomatch=0][order(-Date)]
x1_ftdc=x1_ftdc%>%select(-Date)/100
x2_ftdc=lin_gub_bmybdst_flibfwdspind[x2_ftdc,on=.(Date),nomatch=0][order(-Date)]
x2_ftdc=x2_ftdc%>%select(-Date)/100
X3_ftdc=lin_gub_bmybdst_flibfwdspind[X3_ftdc,on=.(Date),nomatch=0][order(-Date)]
X3_ftdc=X3_ftdc%>%select(-Date)

#historia de curvas de forward ind
#CONSIDERAR LA CURVA GUBERNAMENTAL X1_ORIG_GOV
X3_find=lin_gub_bmybdst_flibfwdspind[X3_find,on=.(Date),nomatch=0][order(-Date)]
X3_find=X3_find%>%select(-Date)

# Pasos para medir el riesgo por el método de Delta-Gamma

Para todos los instrumentos hay que definir y calcular los siguientes elementos:\
1.Historico de factores de riesgo\
2.Vector de precios actual\
3.Derivada con respecto al tiempo evaluada en el precio actual\
4.Primera derivada con respecto a factores de riesgo evaluada y multiplicada con su precio actual\
5.Segunda derivada con respecto a factores de riesgo evaluada y multiplicada con sus precios actuales (al cuadrado)\

##Forwards de tipo de cambio

In [59]:
%%R
#FORWARDS Y/O FUTUROS DE TIPO DE CAMBIO CÁLCULO

################MATRICES DE INTERPOLACION LINEAL ####################
#FORWARDS Y/O FUTUROS DE TIPO DE CAMBIO CÁLCULO

################MATRICES DE INTERPOLACION LINEAL ####################

m=ncol(plazos_fwd)
X1_fwtdc=matrix(0,n,m)
X2_fwtdc=matrix(0,n,m)

for (j in 1:n)
{
  X1_fwtdc[j,]=if(itpl==0){approx(nodos1_ftdc,x1_ftdc[j,],plazos_fwd,rule=2)$y}else{talamb(nodos1_ftdc,x1_ftdc[j,],plazos_fwd)}
  X2_fwtdc[j,]=if(itpl==0){approx(nodos2_ftdc,x2_ftdc[j,],plazos_fwd,rule=2)$y}else{talamb(nodos2_ftdc,x2_ftdc[j,],plazos_fwd)}
  if(trlib==1){X1_fwtdc[j,]=((1+X1_fwtdc[j,])^(plazos_fwd/180)-1)*360/plazos_fwd} #transformación de act 180 a 360
}


futuroTC = function(t,tl,tn,s,k) #t=dias por vencer, tn=tasa nacional para tipo de cambio forward, tl= tasa extranjera pra tipo de cambio forward, S=spot
{
  f=s*((1+tn*t/360)/(1+tl*t/360)) #Se obtiene el tipo de cambio forward
  t(as.numeric((f-k)/(1+t*tn/360))) #Se obtiene el valor del payoff a valor presente con el valor z que define si es largo o corto
}

X3_ftdc=as.matrix(X3_ftdc)

V0_fwtdc=futuroTC(plazos_fwd,X1_fwtdc[1,],X2_fwtdc[1,],X3_ftdc[1,],kst_fwd)*contratos_fwd*nominal_fwd


dfuturoTC_tl = function(t,tl,tn,s,k) #t=dias por vencer, tn=tasa nacional para tipo de cambio forward, tl= tasa extranjera pra tipo de cambio forward, S=spot
{
  f=-s*((1+tn*t/360)/(1+tl*t/360)^2)*(t/360) #Se obtiene el tipo de cambio forward
  t(as.numeric((f)/(1+t*tn/360))) #Se obtiene el valor del payoff a valor presente con el valor z que define si es largo o corto
}

dfuturoTC_tn = function(t,tl,tn,s,k) #t=dias por vencer, tn=tasa nacional para tipo de cambio forward, tl= tasa extranjera pra tipo de cambio forward, S=spot
{
  f=s*((t/360)/(1+tl*t/360)) #Se obtiene el tipo de cambio forward
  f1=s*((1+tn*t/360)/(1+tl*t/360)) #Se obtiene el tipo de cambio forward
  t(as.numeric((f)/(1+t*tn/360)-(f1-k)/(1+t*tn/360)^2*(t/360))) #Se obtiene el valor del payoff a valor presente con el valor z que define si es largo o corto
}

dfuturoTC_s = function(t,tl,tn,s,k) #t=dias por vencer, tn=tasa nacional para tipo de cambio forward, tl= tasa extranjera pra tipo de cambio forward, S=spot
{
  1/(1+t*tl/360) #Se obtiene el valor del payoff a valor presente con el valor z que define si es largo o corto
}

dfuturoTC_t = function(t,tl,tn,s,k) #t=dias por vencer, tn=tasa nacional para tipo de cambio forward, tl= tasa extranjera pra tipo de cambio forward, S=spot
{
  f=s*((1+tn/360)/(1+tl*t/360))-s*((1+tn*t/360)/(1+tl*t/360)^2)*(tl/360) #Se obtiene el tipo de cambio forward
  #t(as.numeric((f)/(1+t*tn/360)-(f-k)/(1+t*tn/360)^2*(tn/360))) #Se obtiene el valor del payoff a valor presente con el valor z que define si es largo o corto
  -(f-k)/(1+t*tn/360)^2*(tn/360)
}

mgamma_futTC= function(t,tl,tn,s,k,contratos_fwd,nominal_fwd)
{
c11= matrix(0,1,length(k))#spot spot
c12= -(((1+tn*t/360)/(1+tl*t/360)^2)*(t/360))/(1+t*tn/360)*contratos_fwd*nominal_fwd#spot tl
c13=  matrix(0,1,length(k))  #spot tn
c21=  -(((1+tn*t/360)/(1+tl*t/360)^2)*(t/360))/(1+t*tn/360)*contratos_fwd*nominal_fwd #tl spot
c22= (2*s*((1+tn*t/360)/(1+tl*t/360)^3)*(t/360)^3)/(1+t*tn/360)*contratos_fwd*nominal_fwd #tl tl
c23= matrix(0,1,length(k)) #tl tn
c31=(((t/360)/(1+tl*t/360))/(1+t*tn/360)-(((1+tn*t/360)/(1+tl*t/360)))/(1+t*tn/360)^2*(t/360))*contratos_fwd*nominal_fwd #tn spot
c32=   (-s*((t/360)^2/(1+tl*t/360)^2)/(1+t*tn/360)-(s*((t/360)/(1+tl*t/360)))/(1+t*tn/360)^2*(t/360))*contratos_fwd*nominal_fwd #tn tl
c33=  (s*((t/360)/(1+tl*t/360))/(1+t*tn/360)^2*(t/360)+2*(s*((1+tn*t/360)/(1+tl*t/360))-k)/(1+t*tn/360)^3*(t/360)^2-(s*((t/360)^2/(1+tl*t/360)))/(1+t*tn/360)^2)*contratos_fwd*nominal_fwd #tn tn
m=length(t)
rbind(cbind(diagv(c11),diagv(c12),diagv(c13)),cbind(diagv(c21),diagv(c22),diagv(c23)),cbind(diagv(c31),diagv(c32),diagv(c33)))
}


X_ftdc=cbind(matrix(X3_ftdc,n,ncol(X1_fwtdc)),X1_fwtdc,X2_fwtdc) #Hist facto riesgo
xp_ftdc=t(as.matrix(X_ftdc[1,])) #Valores actuales X0
mu_ftdc=dfuturoTC_t(plazos_fwd,X1_fwtdc[1,],X2_fwtdc[1,],X3_ftdc[1,],kst_fwd)*contratos_fwd*nominal_fwd #primera derivada con respecto al tiempo evaluada en x0
g_ftdc=cbind(dfuturoTC_s(plazos_fwd,X1_fwtdc[1,],X2_fwtdc[1,],X3_ftdc[1,],kst_fwd)*contratos_fwd*nominal_fwd,dfuturoTC_tl(plazos_fwd,X1_fwtdc[1,],X2_fwtdc[1,],X3_ftdc[1,],kst_fwd)*contratos_fwd*nominal_fwd,dfuturoTC_tn(plazos_fwd,X1_fwtdc[1,],X2_fwtdc[1,],X3_ftdc[1,],kst_fwd)*contratos_fwd*nominal_fwd) #gradiente evaluado en x0
g_ftdctT=xp_ftdc*g_ftdc #gradienete evaluado en x0 por x0
gamma_ftdc=mgamma_futTC(plazos_fwd,X1_fwtdc[1,],X2_fwtdc[1,],X3_ftdc[1,],kst_fwd,contratos_fwd,nominal_fwd) #evaluación de gamma
gamma_ftdctT2=gamma_ftdc*(t(xp_ftdc)%*%xp_ftdc) #evaluación de gamma multiplicado por x^{2}

print(gamma_ftdc)


               [,1]     [,2]      [,3]     [,4]          [,5]        [,6]
 [1,]  0.000000e+00 0.000000   0.00000   0.0000  0.000000e+00   -451.9288
 [2,]  0.000000e+00 0.000000   0.00000   0.0000  0.000000e+00      0.0000
 [3,]  0.000000e+00 0.000000   0.00000   0.0000  0.000000e+00      0.0000
 [4,]  0.000000e+00 0.000000   0.00000   0.0000  0.000000e+00      0.0000
 [5,]  0.000000e+00 0.000000   0.00000   0.0000  0.000000e+00      0.0000
 [6,] -4.519288e+02 0.000000   0.00000   0.0000  0.000000e+00  16573.1751
 [7,]  0.000000e+00 7.235761   0.00000   0.0000  0.000000e+00      0.0000
 [8,]  0.000000e+00 0.000000 -27.96662   0.0000  0.000000e+00      0.0000
 [9,]  0.000000e+00 0.000000   0.00000 118.4785  0.000000e+00      0.0000
[10,]  0.000000e+00 0.000000   0.00000   0.0000 -2.132026e+02      0.0000
[11,] -5.551115e-14 0.000000   0.00000   0.0000  0.000000e+00 -15830.9435
[12,]  0.000000e+00 0.000000   0.00000   0.0000  0.000000e+00      0.0000
[13,]  0.000000e+00 0.000000   0.00000

## Forward de índices

In [60]:
%%R
#FORWARDS Y/O FUTUROS DE ÍNDICES CÁLCULO

################MATRICES DE INTERPOLACION LINEAL ####################

m=ncol(plazos_fwd_ind)
X1_fwind=matrix(0,n,m) #DIVIDENDOS
X2_fwind=matrix(0,n,m)

for (j in 1:n)
{
  #X1_fwind[j,]=if(itpl==0){approx(nodos1_,x1_ftdc[j,],plazos_fwd)$y}else{talamb(nodos1_ftdc,x1_ftdc[j,],plazos_fwd)}
  X2_fwind[j,]=if(itpl==0){approx(nodos_gov,x_orig_gov[j,],plazos_fwd_ind,rule=2)$y}else{talamb(nodos_gov,x_orig_gov[j,],plazos_fwd_ind)}
  #if(trlib==1){X1_fwtdc[j,]=((1+X1_fwtdc[j,])^(plazos_fwd/180)-1)*360/plazos_fwd} #transformación de act 180 a 360
}

X3_find=as.matrix(X3_find)

V0_fwind=futuroTC(plazos_fwd_ind,X1_fwind[1,],X2_fwind[1,],X3_find[1,],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind

X_find=cbind(matrix(X3_find,n,ncol(X1_fwind)),X1_fwind,X2_fwind)
xp_find=t(as.matrix(X_find[1,]))
mu_find=dfuturoTC_t(plazos_fwd_ind,X1_fwind[1,],X2_fwind[1,],X3_find[1,],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind
g_find=cbind(dfuturoTC_s(plazos_fwd_ind,X1_fwind[1,],X2_fwind[1,],X3_find[1,],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind,dfuturoTC_tl(plazos_fwd_ind,X1_fwind[1,],X2_fwind[1,],X3_find[1,],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind,dfuturoTC_tn(plazos_fwd_ind,X1_fwind[1,],X2_fwind[1,],X3_find[1,],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind)
g_findtT=xp_find*g_find
gamma_find=mgamma_futTC(plazos_fwd_ind,X1_fwind[1,],X2_fwind[1,],X3_find[1,],kst_fwd_ind,contratos_fwd_ind,nominal_fwd_ind)
gamma_findtT2=gamma_find*(t(xp_find)%*%xp_find)

print(gamma_find)


      [,1] [,2]      [,3]      [,4]       [,5]          [,6]      [,7]   [,8]
 [1,]    0 0.00   0.00000      -5.0   0.000000  0.000000e+00      0.00  0.000
 [2,]    0 0.00   0.00000       0.0   0.050000  0.000000e+00      0.00  0.000
 [3,]    0 0.00   0.00000       0.0   0.000000 -4.861111e+01      0.00  0.000
 [4,]   -5 0.00   0.00000  416074.2   0.000000  0.000000e+00      0.00  0.000
 [5,]    0 0.05   0.00000       0.0  -2.600464  0.000000e+00      0.00  0.000
 [6,]    0 0.00 -48.61111       0.0   0.000000  3.823556e+08      0.00  0.000
 [7,]    0 0.00   0.00000 -369917.0   0.000000  0.000000e+00 -55058.08  0.000
 [8,]    0 0.00   0.00000       0.0 103.716237  0.000000e+00      0.00 19.416
 [9,]    0 0.00   0.00000       0.0   0.000000 -1.387846e+07      0.00  0.000
         [,9]
 [1,]       0
 [2,]       0
 [3,]       0
 [4,]       0
 [5,]       0
 [6,]       0
 [7,]       0
 [8,]       0
 [9,] 2792347


# Integración de factores y cálculo de varianza-covarianza

In [0]:
%%R
#DIMENSION DE TODOS LOS INSTRUMENTOS
#Son 8 instrumentos financieros (9 si separamos acciones y divisas)
n_if=matrix(0,2,1)
n_if[1]=ncol(X_ftdc) #fwd tdc
n_if[2]=ncol(X_find) #fwd ind

#valor del portafolios

V0_port=cbind(V0_fwtdc, V0_fwind)
V0T_port=sum(V0_port)



#INTEGRACIÓN DE TODOS LOS FACTORES DE RIESGO EN UNA MATRIZ
X_port=cbind(X_ftdc,X_find) #Factores de riesgo del portafolios de 8(9) instrumentos financieros

#Cálculo de variaciones Delta_X DEL PORTAFOLIOS
DeltaX_port=as.matrix(log(X_port[1:(n-1),]/X_port[2:(n),]))
DeltaX_port[is.nan(DeltaX_port)] <- 0 #quitamos NaN
DeltaX_port[is.na(DeltaX_port)] <- 0 #quitamos Na
DeltaX_port[is.infinite(DeltaX_port)] <- 0 #quitamos Na

alpha=0.98 #Nivel de Confianza para las medidas de riesgo

#Cálculo de matriz de varianza covarianza de todos los factores de riesgo del PORTAFOLIO
VarDeltaX_port=as.matrix(cov(DeltaX_port)*(n-1)/(n-2))

mu_port=as.matrix(cbind(mu_ftdc,mu_find)) #derivada con respecto al tiempo
#INTEGRACIÓN DE PRIMERAS DERIVADAS (GRADIENTE)
gt_port=as.matrix(cbind(g_ftdctT,g_findtT)) #derivada con respecto las X's
#validar que ncol(X_port)=ncol(gt_port)


#INTEGRACIÓN DE MATRIZ HESSIANA

gammat_port=matrix(0,sum(n_if),sum(n_if))
gammat_port[1:n_if[1],1:n_if[1]]=gamma_ftdctT2
gammat_port[(sum(n_if[1:1])+1):(sum(n_if[1:2])),(sum(n_if[1:1])+1):(sum(n_if[1:2]))]=gamma_findtT2
#print(gammat_port)
#print(head(DeltaX_s))

# Medición de Riesgo

## Riesgo Forward TdC

###Riesgo a nivel contrato y nivel contrato-factor de riesgo

In [62]:
%%R


#Cálculo de matriz de pérdidas y ganancias FUTUROS TDC

#Cálculo de VaR por contrato
#creación de matrices de VaR, gamma, VaRCF por contrato
DeltaX_ftdc=DeltaX_port[,(1:n_if[1])] 
m=ncol(plazos_fwd)
VaRmargc_ftdc=matrix(0,1,m) #Var por contrato Delta Normal
VaRmargcstd_ftdc=matrix(0,1,m) #Var por contrato estandarizado 
VaRmargcCF_ftdc=matrix(0,1,m) #Var por contrato Cornish-Fisher
CVaRmargc_ftdc=matrix(0,1,m) #CVar por contrato estandarizado
CVaRmargcCF_ftdc=matrix(0,1,m) #CVar por contrato estandarizado

##factores riesgo
#Spot
VaRmargc_fr_ftdc=matrix(0,3,m) #Var por contrato Delta Normal
VaRmargcstd_fr_ftdc=matrix(0,3,m) #Var por contrato estandarizado 
VaRmargcCF_fr_ftdc=matrix(0,3,m) #Var por contrato Cornish-Fisher
CVaRmargc_fr_ftdc=matrix(0,3,m) #CVar por contrato estandarizado
CVaRmargcCF_fr_ftdc=matrix(0,3,m) #CVar por contrato estandarizado
WFR=diagv(c(1,1,1))


for (i in 1:(m))
{
  delta_ftdc=t(g_ftdc[seq(i,m*3,by=m)]) 
  tdelta_ftdc=t(g_ftdctT[seq(i,m*3,by=m)])*xp_ftdc[seq(i,m*3,by=m)]   #delta tilde
  gamma_ftdc=mgamma_futTC(plazos_fwd[i],X1_fwtdc[1,i],X2_fwtdc[1,i],X3_ftdc[1,],kst_fwd[i],contratos_fwd[i],nominal_fwd)  #paso clave
  tgamma_ftdc=xp_ftdc[seq(i,m*3,by=m)]%*%t(xp_ftdc[seq(i,m*3,by=m)])*gamma_ftdc #gamma tilde
  Varmargcaux_ftdc=cov(DeltaX_ftdc[,seq(i,m*3,by=m)])*((n)/(n-1)) #matriz varcov factores de riesgo de cada contrato
  tgammavar_ftdc=(tgamma_ftdc%*%Varmargcaux_ftdc)
  tgammavar2_ftdc=(tgamma_ftdc%*%Varmargcaux_ftdc)%*%(tgamma_ftdc%*%Varmargcaux_ftdc)  
  mudg_ftdc=sum(diag(tgammavar_ftdc))/2+mu_ftdc[i]
  desv_ftdc=sqrt(sum(diag(tgammavar2_ftdc))/2+tdelta_ftdc%*%Varmargcaux_ftdc%*%t(tdelta_ftdc))

  VaRmargc_ftdc[i]=mudg_ftdc+desv_ftdc*qnorm(1-alpha) #VaR Normal por contrato
  CVaRmargc_ftdc[i]=-mudg_ftdc-desv_ftdc*(dnorm(qnorm(1-alpha))/(1-alpha))
  
  tgammavar3_ftdc=tgammavar2_ftdc%*%(tgamma_ftdc%*%Varmargcaux_ftdc)
  asimPmargc_ftdc=sum(diag(tgammavar3_ftdc))+3*tdelta_ftdc%*%Varmargcaux_ftdc%*%tgamma_ftdc%*%Varmargcaux_ftdc%*%t(tdelta_ftdc) #tercer momento
  asimPmargc_ftdc=asimPmargc_ftdc/(desv_ftdc)^3  #coef de asimetría
  VaRmargcstd_ftdc[i]=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_ftdc  #VaR estandarizado
  VaRmargcCF_ftdc[i]=mudg_ftdc+desv_ftdc*VaRmargcstd_ftdc[i] 	    #VaR Cornish-Fisher
  CVaRmargcCF_ftdc[i]=-mudg_ftdc-desv_ftdc*(dnorm(VaRmargcstd_ftdc[i])/(pnorm(VaRmargcstd_ftdc[i])))

#Por factor de riesgo y por contrato

  for (j in 1:3)
  {
    WFRM=matrix(0,3,3)  #PASO CLAVE
    WFRM[j,j]=1  #PASO CLAVE
    delta_ftdc=t(g_ftdc[seq(i,m*3,by=m)])*WFR[j,] #PASO CLAVE
    tdelta_ftdc=t(g_ftdctT[seq(i,m*3,by=m)])*xp_ftdc[seq(i,m*3,by=m)]*WFR[j,]   #delta tilde #PASO CLAVE
    gamma_ftdc=mgamma_futTC(plazos_fwd[i],X1_fwtdc[1,i],X2_fwtdc[1,i],X3_ftdc[1,],kst_fwd[i],contratos_fwd[i],nominal_fwd)*WFRM  #paso clave
    tgamma_ftdc=xp_ftdc[seq(i,m*3,by=m)]%*%t(xp_ftdc[seq(i,m*3,by=m)])*gamma_ftdc #gamma tilde
    Varmargcaux_ftdc=cov(DeltaX_ftdc[,seq(i,m*3,by=m)])*((n)/(n-1)) #matriz varcov factores de riesgo de cada contrato
    tgammavar_ftdc=(tgamma_ftdc%*%Varmargcaux_ftdc)
    tgammavar2_ftdc=(tgamma_ftdc%*%Varmargcaux_ftdc)%*%(tgamma_ftdc%*%Varmargcaux_ftdc)  
    mudg_ftdc=sum(diag(tgammavar_ftdc))/2+mu_ftdc[i]
    desv_ftdc=sqrt(sum(diag(tgammavar2_ftdc))/2+tdelta_ftdc%*%Varmargcaux_ftdc%*%t(tdelta_ftdc))

    VaRmargc_fr_ftdc[j,i]=mudg_ftdc+desv_ftdc*qnorm(1-alpha) #VaR Normal por contrato
    CVaRmargc_fr_ftdc[j,i]=-mudg_ftdc-desv_ftdc*(dnorm(qnorm(1-alpha))/(1-alpha))
    
    tgammavar3_ftdc=tgammavar2_ftdc%*%(tgamma_ftdc%*%Varmargcaux_ftdc)
    asimPmargc_ftdc=sum(diag(tgammavar3_ftdc))+3*tdelta_ftdc%*%Varmargcaux_ftdc%*%tgamma_ftdc%*%Varmargcaux_ftdc%*%t(tdelta_ftdc) #tercer momento
    asimPmargc_ftdc=asimPmargc_ftdc/(desv_ftdc)^3  #coef de asimetría
    VaRmargcstd_fr_ftdc[j,i]=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_ftdc  #VaR estandarizado
    VaRmargcCF_fr_ftdc[j,i]=mudg_ftdc+desv_ftdc*VaRmargcstd_fr_ftdc[j,i] 	    #VaR Cornish-Fisher
    CVaRmargcCF_fr_ftdc[j,i]=-mudg_ftdc-desv_ftdc*(dnorm(VaRmargcstd_fr_ftdc[j,i])/(pnorm(VaRmargcstd_fr_ftdc[j,i])))
  }



}


print(VaRmargc_ftdc)
print(VaRmargcCF_ftdc)
print(CVaRmargcCF_ftdc)

print(VaRmargcCF_fr_ftdc)
print(CVaRmargcCF_fr_ftdc)


          [,1]     [,2]     [,3]      [,4]      [,5]
[1,] -1996.235 -1216.79 -1212.96 -1908.756 -1804.069
          [,1]     [,2]     [,3]      [,4]     [,5]
[1,] -1996.234 -1216.79 -1212.96 -1908.756 -1804.07
         [,1]      [,2]      [,3]     [,4]      [,5]
[1,] -2353.58 -1434.303 -1429.995 -2249.79 -2126.928
              [,1]          [,2]          [,3]          [,4]          [,5]
[1,] -1.996248e+03 -1.216790e+03 -1.212961e+03 -1908.7575583 -1.804067e+03
[2,] -5.659658e-02 -9.609434e-03  8.399578e-02    -0.1253017  7.728302e-02
[3,] -5.082858e-01 -1.396148e-02  7.685556e-02    -0.1928708 -8.273985e-02
              [,1]          [,2]          [,3]          [,4]          [,5]
[1,] -2353.5967135 -1.434303e+03 -1.429996e+03 -2.249791e+03 -2126.9246487
[2,]    -0.5393461  6.454755e-03 -8.961060e-02  5.565710e-02    -0.2460118
[3,]    -1.0693398  1.324666e-03 -9.802715e-02 -2.400555e-02    -0.4344925


###Riesgo a nivel total y total factor de riesgo

In [63]:
%%R


#RIESGO TOTAL FORWARDS DE TIPO DE CAMBIO

VarDeltaX_ftdc=cov(DeltaX_ftdc)*(n/(n-1))

tgammavar_ftdc=(gamma_ftdctT2%*%VarDeltaX_ftdc)
tgammavar2_ftdc=(tgammavar_ftdc)%*%(tgammavar_ftdc) 
mudg_ftdc=sum(diag(tgammavar_ftdc))/2+sum(mu_ftdc)
desv_ftdc=sqrt(sum(diag(tgammavar2_ftdc))/2+g_ftdctT%*%VarDeltaX_ftdc%*%t(g_ftdctT))

VaRT_ftdc_DGM=(mudg_ftdc)+(desv_ftdc)*qnorm(1-alpha)  #VaR Normal
CVaRT_ftdc_DGM=-(mudg_ftdc)-(desv_ftdc)*dnorm(qnorm(alpha))/(1-alpha)


###VaR Cornish Fisher
tgammavar3_ftdc=tgammavar2_ftdc%*%(tgammavar_ftdc)
asimP_ftdc=sum(diag(tgammavar3_ftdc))+3*g_ftdctT%*%VarDeltaX_ftdc%*%gamma_ftdctT2%*%VarDeltaX_ftdc%*%t(g_ftdctT) #gamma tercer momento
asimP_ftdc=asimP_ftdc/(desv_ftdc)^3  #coef de asimetría
VaRstd_ftdc=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_ftdc  #VaR estandarizado
VaRT_ftdc_DGM_CF=(mudg_ftdc)+(desv_ftdc)*VaRstd_ftdc #VaR cf
CVaRT_ftdc_DGM_CF=-(mudg_ftdc)-(desv_ftdc)*dnorm(VaRstd_ftdc)/(pnorm(VaRstd_ftdc))

print(VaRT_ftdc_DGM)
print(CVaRT_ftdc_DGM)
print(VaRT_ftdc_DGM_CF)
print(CVaRT_ftdc_DGM_CF)

VaRT_fr_ftdc_DGM=matrix(0,1,3)
CVaRT_fr_ftdc_DGM=matrix(0,1,3)
VaRT_fr_ftdc_DGM_CF=matrix(0,1,3)
CVaRT_fr_ftdc_DGM_CF=matrix(0,1,3)

WFRT=matrix(0,3,(m*3))
# WFRT[1,1:m]=matrix(1,1,m)
# WFRT[2,(m+1):(2*m)]=matrix(1,1,m)
# WFRT[3,(2*m+1):(3*m)]=matrix(1,1,m)
print(WFRT)

#RIESGO TOTAL POR FACTOR DE RIESGO DE FORWARDS DE TIPO DE CAMBIO

for (j in (1:3))
{
WFRT[j,((j-1)*m+1):(j*m)]=matrix(1,1,m)    #Matriz de diseño para 
WFRM=matrix(0,(m*3),(m*3))  #PASO CLAVE
WFRM[((j-1)*m+1):(m*j),((j-1)*m+1):(m*j)]=matrix(1,m,m)  #PASO CLAVE    
gamma_ftdctT2_fr=gamma_ftdctT2*WFRM #evaluación de gamma multiplicado por x^{2}
g_ftdctT_fr=g_ftdctT*WFRT[j,]
tgammavar_ftdc=(gamma_ftdctT2_fr%*%VarDeltaX_ftdc)
tgammavar2_ftdc=(tgammavar_ftdc)%*%(tgammavar_ftdc) 
mudg_ftdc=sum(diag(tgammavar_ftdc))/2+sum(mu_ftdc)
desv_ftdc=sqrt(sum(diag(tgammavar2_ftdc))/2+g_ftdctT_fr%*%VarDeltaX_ftdc%*%t(g_ftdctT_fr))
VaRT_fr_ftdc_DGM[j]=(mudg_ftdc)+(desv_ftdc)*qnorm(1-alpha) #VaR Normal
CVaRT_fr_ftdc_DGM[j]=-(mudg_ftdc)-(desv_ftdc)*dnorm(qnorm(alpha))/(1-alpha)


###VaR Cornish Fisher
tgammavar3_ftdc=tgammavar2_ftdc%*%(tgammavar_ftdc)
asimP_ftdc=sum(diag(tgammavar3_ftdc))+3*g_ftdctT_fr%*%VarDeltaX_ftdc%*%gamma_ftdctT2_fr%*%VarDeltaX_ftdc%*%t(g_ftdctT_fr) #gamma tercer momento
 print(asimP_ftdc)
asimP_ftdc=asimP_ftdc/(desv_ftdc)^3  #coef de asimetría
 
VaRstd_ftdc=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_ftdc  #VaR estandarizado
print(VaRstd_ftdc)
VaRT_fr_ftdc_DGM_CF[j]=(mudg_ftdc)+(desv_ftdc)*VaRstd_ftdc #VaR cf
CVaRT_fr_ftdc_DGM_CF[j]=-(mudg_ftdc)-(desv_ftdc)*dnorm(VaRstd_ftdc)/(pnorm(VaRstd_ftdc))
    
}


print(VaRT_fr_ftdc_DGM)
print(CVaRT_fr_ftdc_DGM)
print(VaRT_fr_ftdc_DGM_CF)
print(CVaRT_fr_ftdc_DGM_CF)




# cbind(VaRTotal_fwtdc,sum(V0_fwtdc), VaRCont_fwtdc, V0_fwtdc)
# cbind(CVaRTotal_fwtdc,sum(V0_fwtdc), CVaRCont_fwtdc, V0_fwtdc)
# cbind(VaRTotal_fwtdc,VaRTotalfr1_fwtdc,VaRTotalfr2_fwtdc,VaRTotalfr3_fwtdc)
# cbind(CVaRTotal_fwtdc,CVaRTotalfr1_fwtdc,CVaRTotalfr2_fwtdc,CVaRTotalfr3_fwtdc)


         [,1]
[1,] -97.5631
          [,1]
[1,] -115.7793
          [,1]
[1,] -97.56424
          [,1]
[1,] -115.7803
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    0    0    0    0    0    0    0    0    0     0     0     0     0     0
[2,]    0    0    0    0    0    0    0    0    0     0     0     0     0     0
[3,]    0    0    0    0    0    0    0    0    0     0     0     0     0     0
     [,15]
[1,]     0
[2,]     0
[3,]     0
     [,1]
[1,]    0
          [,1]
[1,] -2.053749
           [,1]
[1,] 0.04199745
          [,1]
[1,] -2.052951
             [,1]
[1,] -0.009252664
          [,1]
[1,] -2.053789
          [,1]      [,2]      [,3]
[1,] -97.55104 -5.896589 -9.869062
          [,1]      [,2]     [,3]
[1,] -115.7655 -7.727257 -12.4081
          [,1]     [,2]      [,3]
[1,] -97.55104 -5.89416 -9.869262
          [,1]      [,2]      [,3]
[1,] -115.7655 -7.725097 -12.40828


## Riesgo Forwards Índices

###Riesgo a nivel contrato y contrato-factor de riesgo

In [64]:
%%R

DeltaX_find=DeltaX_port[,(n_if[1]+1):(sum(n_if[1:2]))] 
m=ncol(plazos_fwd_ind)
VaRmargc_find=matrix(0,1,m) #Var por contrato Delta Normal
VaRmargcstd_find=matrix(0,1,m) #Var por contrato estandarizado 
VaRmargcCF_find=matrix(0,1,m) #Var por contrato Cornish-Fisher
CVaRmargc_find=matrix(0,1,m) #CVar por contrato estandarizado
CVaRmargcCF_find=matrix(0,1,m) #CVar por contrato estandarizado

##factores riesgo
#Spot
VaRmargc_fr_find=matrix(0,3,m) #Var por contrato Delta Normal
VaRmargcstd_fr_find=matrix(0,3,m) #Var por contrato estandarizado 
VaRmargcCF_fr_find=matrix(0,3,m) #Var por contrato Cornish-Fisher
CVaRmargc_fr_find=matrix(0,3,m) #CVar por contrato estandarizado
CVaRmargcCF_fr_find=matrix(0,3,m) #CVar por contrato estandarizado
WFR=diagv(c(1,1,1))


for (i in 1:(m))
{
  delta_find=t(g_find[seq(i,m*3,by=m)]) 
  tdelta_find=t(g_findtT[seq(i,m*3,by=m)])*xp_find[seq(i,m*3,by=m)]   #delta tilde
  gamma_find=mgamma_futTC(plazos_fwd_ind[i],X1_fwind[1,i],X2_fwind[1,i],X3_find[1,],kst_fwd_ind[i],contratos_fwd_ind[i],nominal_fwd_ind)  #paso clave
  tgamma_find=xp_find[seq(i,m*3,by=m)]%*%t(xp_find[seq(i,m*3,by=m)])*gamma_find #gamma tilde
  Varmargcaux_find=cov(DeltaX_find[,seq(i,m*3,by=m)])*((n)/(n-1)) #matriz varcov factores de riesgo de cada contrato
  tgammavar_find=(tgamma_find%*%Varmargcaux_find)
  tgammavar2_find=(tgamma_find%*%Varmargcaux_find)%*%(tgamma_find%*%Varmargcaux_find)  
  mudg_find=sum(diag(tgammavar_find))/2+mu_find[i]
  desv_find=sqrt(sum(diag(tgammavar2_find))/2+tdelta_find%*%Varmargcaux_find%*%t(tdelta_find))

  VaRmargc_find[i]=mudg_find+desv_find*qnorm(1-alpha) #VaR Normal por contrato
  CVaRmargc_find[i]=-mudg_find-desv_find*(dnorm(qnorm(1-alpha))/(1-alpha))
  
  tgammavar3_find=tgammavar2_find%*%(tgamma_find%*%Varmargcaux_find)
  asimPmargc_find=sum(diag(tgammavar3_find))+3*tdelta_find%*%Varmargcaux_find%*%tgamma_find%*%Varmargcaux_find%*%t(tdelta_find) #tercer momento
  asimPmargc_find=asimPmargc_find/(desv_find)^3  #coef de asimetría
  VaRmargcstd_find[i]=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_find  #VaR estandarizado
  VaRmargcCF_find[i]=mudg_find+desv_find*VaRmargcstd_find[i] 	    #VaR Cornish-Fisher
  CVaRmargcCF_find[i]=-mudg_find-desv_find*(dnorm(VaRmargcstd_find[i])/(pnorm(VaRmargcstd_find[i])))

#Por factor de riesgo y por contrato

  for (j in 1:3)
  {
    WFRM=matrix(0,3,3)  #PASO CLAVE
    WFRM[j,j]=1  #PASO CLAVE
    delta_find=t(g_find[seq(i,m*3,by=m)])*WFR[j,] #PASO CLAVE
    tdelta_find=t(g_findtT[seq(i,m*3,by=m)])*xp_find[seq(i,m*3,by=m)]*WFR[j,]   #delta tilde #PASO CLAVE
  	gamma_find=mgamma_futTC(plazos_fwd_ind[i],X1_fwind[1,i],X2_fwind[1,i],X3_find[1,],kst_fwd_ind[i],contratos_fwd_ind[i],nominal_fwd_ind)  #paso clave
    tgamma_find=xp_find[seq(i,m*3,by=m)]%*%t(xp_find[seq(i,m*3,by=m)])*gamma_find #gamma tilde
    Varmargcaux_find=cov(DeltaX_find[,seq(i,m*3,by=m)])*((n)/(n-1)) #matriz varcov factores de riesgo de cada contrato
    tgammavar_find=(tgamma_find%*%Varmargcaux_find)
    tgammavar2_find=(tgamma_find%*%Varmargcaux_find)%*%(tgamma_find%*%Varmargcaux_find)  
    mudg_find=sum(diag(tgammavar_find))/2+mu_find[i]
    desv_find=sqrt(sum(diag(tgammavar2_find))/2+tdelta_find%*%Varmargcaux_find%*%t(tdelta_find))

    VaRmargc_fr_find[j,i]=mudg_find+desv_find*qnorm(1-alpha) #VaR Normal por contrato
    CVaRmargc_fr_find[j,i]=-mudg_find-desv_find*(dnorm(qnorm(1-alpha))/(1-alpha))
    
    tgammavar3_find=tgammavar2_find%*%(tgamma_find%*%Varmargcaux_find)
    asimPmargc_find=sum(diag(tgammavar3_find))+3*tdelta_find%*%Varmargcaux_find%*%tgamma_find%*%Varmargcaux_find%*%t(tdelta_find) #tercer momento
    asimPmargc_find=asimPmargc_find/(desv_find)^3  #coef de asimetría
    VaRmargcstd_fr_find[j,i]=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_find  #VaR estandarizado
    VaRmargcCF_fr_find[j,i]=mudg_find+desv_find*VaRmargcstd_fr_find[j,i] 	    #VaR Cornish-Fisher
    CVaRmargcCF_fr_find[j,i]=-mudg_find-desv_find*(dnorm(VaRmargcstd_fr_find[j,i])/(pnorm(VaRmargcstd_fr_find[j,i])))
  }



}


print(VaRmargc_find)
print(VaRmargcCF_find)
print(CVaRmargcCF_find)

print(VaRmargcCF_fr_find)
print(CVaRmargcCF_fr_find)

           [,1]      [,2]       [,3]
[1,] -157731366 -63092553 -157731383
           [,1]      [,2]       [,3]
[1,] -157731366 -63092553 -157731383
           [,1]      [,2]       [,3]
[1,] -185929730 -74371884 -185929751
              [,1]          [,2]          [,3]
[1,] -1.577314e+08 -6.309255e+07 -1.577314e+08
[2,]  1.005154e+01 -3.390092e+00  7.530439e+00
[3,] -1.244799e+01 -3.568857e+00 -2.389383e+02
              [,1]          [,2]          [,3]
[1,] -1.859297e+08 -7.437188e+07 -1.859297e+08
[2,] -1.015558e+01  3.390087e+00 -1.733692e+01
[3,] -3.667367e+01  3.179363e+00 -3.086198e+02


###Riesgo a nivel total y total factor de riesgo

In [65]:
%%R


#RIESGO TOTAL FORWARDS DE TIPO DE CAMBIO

VarDeltaX_find=cov(DeltaX_find)*(n/(n-1))

tgammavar_find=(gamma_findtT2%*%VarDeltaX_find)
tgammavar2_find=(tgammavar_find)%*%(tgammavar_find) 
#media
mudg_find=sum(diag(tgammavar_find))/2+sum(mu_find)
#desviación estándar
desv_find=sqrt(sum(diag(tgammavar2_find))/2+g_findtT%*%VarDeltaX_find%*%t(g_findtT))

VaRT_find_DGM=(mudg_find)+(desv_find)*qnorm(1-alpha)  #VaR Normal
CVaRT_find_DGM=-(mudg_find)-(desv_find)*dnorm(qnorm(alpha))/(1-alpha)


###VaR Cornish Fisher
tgammavar3_find=tgammavar2_find%*%(tgammavar_find)
asimP_find=sum(diag(tgammavar3_find))+3*g_findtT%*%VarDeltaX_find%*%gamma_findtT2%*%VarDeltaX_find%*%t(g_findtT) #gamma tercer momento
asimP_find=asimP_find/(desv_find)^3  #coef de asimetría
VaRstd_find=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_find  #VaR estandarizado
VaRT_find_DGM_CF=(mudg_find)+(desv_find)*VaRstd_find #VaR cf
CVaRT_find_DGM_CF=-(mudg_find)-(desv_find)*dnorm(VaRstd_find)/(pnorm(VaRstd_find))

print(VaRT_find_DGM)
print(CVaRT_find_DGM)
print(VaRT_find_DGM_CF)
print(CVaRT_find_DGM_CF)

VaRT_fr_find_DGM=matrix(0,1,3)
CVaRT_fr_find_DGM=matrix(0,1,3)
VaRT_fr_find_DGM_CF=matrix(0,1,3)
CVaRT_fr_find_DGM_CF=matrix(0,1,3)

WFRT=matrix(0,3,(m*3))
# WFRT[1,1:m]=matrix(1,1,m)
# WFRT[2,(m+1):(2*m)]=matrix(1,1,m)
# WFRT[3,(2*m+1):(3*m)]=matrix(1,1,m)
print(WFRT)

#RIESGO TOTAL POR FACTOR DE RIESGO DE FORWARDS DE TIPO DE CAMBIO

for (j in (1:3))
{
WFRT[j,((j-1)*m+1):(j*m)]=matrix(1,1,m)    #Matriz de diseño para 
WFRM=matrix(0,(m*3),(m*3))  #PASO CLAVE
WFRM[((j-1)*m+1):(m*j),((j-1)*m+1):(m*j)]=matrix(1,m,m)  #PASO CLAVE    
gamma_findtT2_fr=gamma_findtT2*WFRM #evaluación de gamma multiplicado por x^{2}
g_findtT_fr=g_findtT*WFRT[j,]
tgammavar_find=(gamma_findtT2_fr%*%VarDeltaX_find)
tgammavar2_find=(tgammavar_find)%*%(tgammavar_find) 
mudg_find=sum(diag(tgammavar_find))/2+sum(mu_find)
desv_find=sqrt(sum(diag(tgammavar2_find))/2+g_findtT_fr%*%VarDeltaX_find%*%t(g_findtT_fr))
VaRT_fr_find_DGM[j]=(mudg_find)+(desv_find)*qnorm(1-alpha) #VaR Normal
CVaRT_fr_find_DGM[j]=-(mudg_find)-(desv_find)*dnorm(qnorm(alpha))/(1-alpha)


###VaR Cornish Fisher
tgammavar3_find=tgammavar2_find%*%(tgammavar_find)
asimP_find=sum(diag(tgammavar3_find))+3*g_findtT_fr%*%VarDeltaX_find%*%gamma_findtT2_fr%*%VarDeltaX_find%*%t(g_findtT_fr) #gamma tercer momento
asimP_find=asimP_find/(desv_find)^3  #coef de asimetría
 
VaRstd_find=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_find  #VaR estandarizado
VaRT_fr_find_DGM_CF[j]=(mudg_find)+(desv_find)*VaRstd_find #VaR cf
CVaRT_fr_find_DGM_CF[j]=-(mudg_find)-(desv_find)*dnorm(VaRstd_find)/(pnorm(VaRstd_find))
    
}


print(VaRT_fr_find_DGM)
print(CVaRT_fr_find_DGM)
print(VaRT_fr_find_DGM_CF)
print(CVaRT_fr_find_DGM_CF)


          [,1]
[1,] -6682.105
          [,1]
[1,] -7914.452
          [,1]
[1,] -6679.737
          [,1]
[1,] -7912.348
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9]
[1,]    0    0    0    0    0    0    0    0    0
[2,]    0    0    0    0    0    0    0    0    0
[3,]    0    0    0    0    0    0    0    0    0
     [,1]
[1,]    0
          [,1]
[1,] -2.053749
     [,1]
[1,]    0
     [,1]
[1,]  NaN
         [,1]
[1,] 33849343
          [,1]
[1,] -2.042834
          [,1]     [,2]     [,3]
[1,] -6052.237 13.27269 -2415.99
         [,1]      [,2]      [,3]
[1,] -7163.14 -13.27269 -2885.665
          [,1] [,2]      [,3]
[1,] -6052.237  NaN -2403.058
         [,1] [,2]      [,3]
[1,] -7163.14  NaN -2874.174


# Riesgo integral

In [0]:
%%R


#CÁLCULO DE VAR Y CVAR A NIVEL PORTAFOLIO
tgammavar_port=(gammat_port%*%VarDeltaX_port)
tgammavar2_port=(tgammavar_port)%*%(tgammavar_port) 
VaRT_port_DGM=(sum(diag(tgammavar_port))/2+sum(mu_port))-(sqrt(sum(diag(tgammavar2_port))/2+gt_port%*%VarDeltaX_port%*%t(gt_port)))*qnorm(alpha) #VaR Normal
CVaRT_port_DGM=(sum(diag(tgammavar_port))/2+sum(mu_port))-(sqrt(sum(diag(tgammavar2_port))/2+gt_port%*%VarDeltaX_port%*%t(gt_port)))*dnorm(qnorm(alpha))/(1-alpha)

tgammavar2_port[(sum(n_if[1:5])+1):(sum(n_if[1:6])),(sum(n_if[1:5])+1):(sum(n_if[1:6]))]

###VaR Cornish Fisher
tgammavar3_port=tgammavar2_port%*%(tgammavar_port)
asimP_port=sum(diag(tgammavar3_port))+3*gt_port%*%VarDeltaX_port%*%gammat_port%*%VarDeltaX_port%*%t(gt_port) #gamma tercer momento
asimP_port=asimP_port/(sqrt(sum(diag(tgammavar2_port))/2+gt_port%*%VarDeltaX_port%*%t(gt_port)))^3  #coef de asimetría
VaRstd_port=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_port  #VaR estandarizado
VaRT_port_DGM_CF=(sum(diag(tgammavar_port))/2+sum(mu_port))+(sqrt(sum(diag(tgammavar2_port))/2+gt_port%*%VarDeltaX_port%*%t(gt_port)))*VaRstd_port #VaR cf
CVaRT_port_DGM_CF=(sum(diag(tgammavar_port))/2+sum(mu_port))-(sqrt(sum(diag(tgammavar2_port))/2+gt_port%*%VarDeltaX_port%*%t(gt_port)))*dnorm(VaRstd_port)/(pnorm(VaRstd_port))


VaRT_port_DGM/V0T_port
CVaRT_port_DGM/V0T_port
VaRT_port_DGM_CF/V0T_port
CVaRT_port_DGM_CF/V0T_port

print(VaRTotal_Port)
print(CVaRTotal_Port)
print(V0T_port)

       2% 
-7450.107 
[1] -8531.94
[1] 88772
